# **1 - Tarefa e Dados**
Descreva a tarefa escolhida e os dados. Escreva código que leia os dados e calcule e imprima quantas instâncias os dados têm. Também, seu código deve calcular a média de tokens por instância, isto é, quantos tokens, na média cada documento do seu conjunto de dados possui. Imprima esse único número.


A corpus escolhido foi o UTL que é um corpus com críticas de filmes e apps coletadas automaticamente de sites. As classes são: positiva ou negativa. Assim o usuário pode ter gostado ou não gostado do produto. Referência: https://github.com/RogerFig/UTLCorpus 


In [ ]:
# montando o drive do google onde o corpus se encontra
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# conferir que o Google Drive está devidamente montado (deve aparecer a pasta 'drive')
%ls -la

total 20
drwxr-xr-x 1 root root 4096 Nov 18 21:16 ./
drwxr-xr-x 1 root root 4096 Nov 18 21:13 ../
drwxr-xr-x 1 root root 4096 Nov 13 17:34 .config/
drwx------ 5 root root 4096 Nov 18 21:16 drive/
drwxr-xr-x 1 root root 4096 Nov 13 17:33 sample_data/


In [ ]:
# Entrando no diretório
%cd drive/My Drive/Colab Notebooks/NLP/trabalho

/content/drive/My Drive/Colab Notebooks/NLP/trabalho


In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Herica\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [2]:
import pandas as pd

In [75]:
# abrindo o arquivo.
df_util = pd.read_csv(r'C:\Users\Herica\Documents\Herica\Pos\GitHub\base\NLP\UTLCorpus.csv')
df_util.head(3)

,Unnamed: 0,domain,object,author,text,likes,unlikes,stars,date,collect_date,replies,favorite,want_see,recommend,see,internal_id
0,0,movies,007-cassino-royale-t23,0,Um dos melhores do 007,0,0,3.5,8 anos atrás,20-04-2019 06:20,0.0,0.0,0.0,NaN,1.0,0
1,1,movies,007-cassino-royale-t23,1,"assisti só pela metade..do meio ao fim, o film...",0,0,4.0,8 anos atrás,20-04-2019 06:20,0.0,0.0,0.0,NaN,1.0,1
2,2,movies,007-cassino-royale-t23,2,foi um dos filmes mais violentos q já vi. mas ...,0,0,4.0,8 anos atrás,20-04-2019 06:20,0.0,0.0,0.0,NaN,1.0,2


# Analisando os dados

## Verificando e deletando registros nulos

In [47]:
df_util.isnull().sum()

Unnamed: 0            0
domain                0
object                0
author                0
text               3152
likes                 0
unlikes               0
stars                 0
date                  0
collect_date          0
replies         1041738
favorite        1041738
want_see        1041738
recommend       2881589
see             1041738
internal_id           0
dtype: int64

In [76]:
# deletando os registros nulos
qtde_anterior_nulos = df_util.shape[0]
df_util.dropna(subset=['text'], inplace = True)
qtde_pos_exclusao_nulos = df_util.shape[0]
print ("Antes:", qtde_anterior_nulos, 'Depois:', qtde_pos_exclusao_nulos, 'Diferença:', qtde_anterior_nulos-qtde_pos_exclusao_nulos)

Antes: 2881589 Depois: 2878437 Diferença: 3152


In [50]:
df_util['domain'].value_counts()

movies    1838895
apps      1039542
Name: domain, dtype: int64

## Encontrando os registros que não possuem like e nem unlike e log após, excluí-los

In [58]:
df_util[['likes','unlikes']]

,likes,unlikes
0,0,0
1,0,0
2,0,0
3,0,0
4,1,0
...,...,...
2881584,79,0
2881585,9,0
2881586,3,0
2881587,1,0


In [54]:
df_util['likes'].describe()

count    2.878437e+06
mean     3.278012e+00
std      5.201631e+01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      2.000000e+00
max      2.014600e+04
Name: likes, dtype: float64

In [57]:
# valores unicos do likes 
df_util['likes'].unique()

array([   0,    1,    2, ..., 3449, 2392,  945], dtype=int64)

In [59]:
df_util['likes'].value_counts()

0       1669268
1        465005
2        244818
3        142268
4         88406
         ...   
3076          1
1029          1
4468          1
1034          1
4095          1
Name: likes, Length: 1505, dtype: int64

In [106]:
# Todas os valores são zero. Concluo que devo utilizar somente a feature likes, sendo que a qtde de likes varia de 0 a 5, sendo que zero quer dizer que não possui like
df_util['unlikes'].describe()

count    2442966.0
mean           0.0
std            0.0
min            0.0
25%            0.0
50%            0.0
75%            0.0
max            0.0
Name: unlikes, dtype: float64

In [ ]:
df_util.query ('likes == 0 and unlikes == 0')

In [ ]:
# Visualizando que todos os likes igual a zero, tem unlikes igual a zero, logo a feature unlikes é descartável
df_util[(df_util['likes'] != 0) & (df_util['unlikes'] != 0)]

## Verificando registros duplicados do par object x author


In [62]:
# encontrando os registros duplicados no ID
df_duplicado = df_util.groupby(['author','object']).size().reset_index(name='count')
df_duplicado.query('count > 1')

,author,object,count
4,0,127-horas-t24843,2
5,0,2001-uma-odisseia-no-espaco-t1422,3
7,0,21-gramas-t4077,2
15,0,a-arvore-da-vida-t14320,4
34,0,a-era-do-gelo-2-t1232,2
...,...,...,...
2436343,534373,radiotime.player,2
2436416,534446,radiotime.player,2
2436468,534498,radiotime.player,2
2441254,539289,wp.wattpad,2


In [11]:
# conforme observado acima, há repetição de registros
# visualizando um dos registros
df_util[df_util['object'] == 'cisne-negro-t16349']

,Unnamed: 0,domain,object,author,text,likes,unlikes,stars,date,collect_date,replies,favorite,want_see,recommend,see,internal_id
464338,464338,movies,cisne-negro-t16349,4288,no filme temos a visão dela do que acontece......,0,0,5.0,8 anos atrás,18-04-2019 19:00,0.0,1.0,0.0,NaN,1.0,464338
464339,464339,movies,cisne-negro-t16349,29909,"Achei genial a abordagem psicológica do filme,...",0,0,4.5,8 anos atrás,18-04-2019 19:00,0.0,1.0,0.0,NaN,1.0,464339
464340,464340,movies,cisne-negro-t16349,42302,Filme sensacional!!Superou minhas expectativas...,0,0,5.0,8 anos atrás,18-04-2019 19:00,0.0,0.0,0.0,NaN,1.0,464340
464341,464341,movies,cisne-negro-t16349,10376,Perturbador e Perfeito!,0,0,0.0,8 anos atrás,18-04-2019 19:00,0.0,0.0,0.0,NaN,1.0,464341
464342,464342,movies,cisne-negro-t16349,74353,"Foda a garota ficar paranóica, ainda mais quan...",0,0,4.5,8 anos atrás,18-04-2019 19:00,0.0,1.0,0.0,NaN,1.0,464342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472172,472172,movies,cisne-negro-t16349,13272,"Simplesmente ""foi perfeito"".",0,0,5.0,8 anos atrás,18-04-2019 19:00,0.0,0.0,0.0,NaN,1.0,472172
472173,472173,movies,cisne-negro-t16349,3162,A cena da transformação me deu até arrepio! sé...,1,0,5.0,8 anos atrás,18-04-2019 19:00,0.0,1.0,0.0,NaN,1.0,472173
472174,472174,movies,cisne-negro-t16349,4288,Briiilhante geeniaal!adorei o filme se ela não...,0,0,5.0,8 anos atrás,18-04-2019 19:00,0.0,1.0,0.0,NaN,1.0,472174
472175,472175,movies,cisne-negro-t16349,54962,Darren Aronofsky empresta ao roteiro imperfeit...,0,0,4.0,8 anos atrás,18-04-2019 19:00,0.0,1.0,0.0,NaN,1.0,472175


In [88]:
# verificando alguns dos valores duplicados de like de um authos/object
#df_util.query('author == 0 and object == "127-horas-t24843"') # registros iguais
#df_util.query('author == 0 and object == "2001-uma-odisseia-no-espaco-t1422"') # registros de data diferente e com likes diferentes
df_util.query('author == 0 and object == "a-arvore-da-vida-t14320"') # possui registros de data diferentes e likes diferentes. likes:0,4,5,5

,Unnamed: 0,domain,object,author,text,likes,unlikes,stars,date,collect_date,replies,favorite,want_see,recommend,see,internal_id
31951,31951,movies,a-arvore-da-vida-t14320,0,Isso aqui tá bombando em spoilers.,0,0,5.0,7 anos atrás,20-04-2019 05:40,0.0,1.0,0.0,NaN,1.0,31951
33686,33686,movies,a-arvore-da-vida-t14320,0,Cinema para a eternidade.,4,0,5.0,6 anos atrás,20-04-2019 05:40,0.0,1.0,0.0,NaN,1.0,33686
31991,31991,movies,a-arvore-da-vida-t14320,0,"É o filme do Malick que mais me tocou, é o fil...",5,0,5.0,7 anos atrás,20-04-2019 05:40,7.0,1.0,0.0,NaN,1.0,31991
34011,34011,movies,a-arvore-da-vida-t14320,0,Eu jamais entenderei o porque que as pessoas a...,5,0,5.0,5 anos atrás,20-04-2019 05:39,1.0,1.0,0.0,NaN,1.0,34011


In [89]:
# Eliminando os registros com repetição, mantendo os que possuem a maior qtde de likes
qtde_anterior_duplicados = df_util.shape[0]

df_util = df_util.sort_values(["author", "object", "likes"], ascending=[True,True,True])
df_util.drop_duplicates(subset=['author', 'object'], keep='last', inplace = True)


qtde_pos_exclusao_duplicados = df_util.shape[0]
print ('antes:', qtde_anterior_duplicados, 'depois:', qtde_pos_exclusao_duplicados, 'diferença:', qtde_anterior_duplicados-qtde_pos_exclusao_duplicados)

antes: 2878437 depois: 2442966 diferença: 435471


In [90]:
# validando a exclusão das duplicatas
#df_util.query('author == 0 and object == "127-horas-t24843"') # registros iguais
#df_util.query('author == 0 and object == "2001-uma-odisseia-no-espaco-t1422"') # registros de data diferente e com likes diferentes
df_util.query('author == 0 and object == "a-arvore-da-vida-t14320"') # possui registros de data diferentes e likes diferentes. likes:0,4,5,5

,Unnamed: 0,domain,object,author,text,likes,unlikes,stars,date,collect_date,replies,favorite,want_see,recommend,see,internal_id
34011,34011,movies,a-arvore-da-vida-t14320,0,Eu jamais entenderei o porque que as pessoas a...,5,0,5.0,5 anos atrás,20-04-2019 05:39,1.0,1.0,0.0,NaN,1.0,34011


In [105]:
# contado e mostrando quais são os filmes únicos
registros_unicos = df_util['object'].unique()
print ('qtde de registros:', len(registros_unicos), '\nvalores unicos:', registros_unicos)

qtde de registros: 4436 
valores unicos: ['007-cassino-royale-t23' '007-operacao-skyfall-t31713'
 '10-coisas-que-eu-odeio-em-voce-t1964' ... 'um-caso-de-familia-t202015'
 'um-homem-e-sua-paixao-t108598' 'voce-nunca-mais-me-vera-t101629']


In [115]:
df_util.head(5)

,Unnamed: 0,domain,object,author,text,likes,unlikes,stars,date,collect_date,replies,favorite,want_see,recommend,see,internal_id
0,0,movies,007-cassino-royale-t23,0,Um dos melhores do 007,0,0,3.5,8 anos atrás,20-04-2019 06:20,0.0,0.0,0.0,NaN,1.0,0
2489,2489,movies,007-operacao-skyfall-t31713,0,Javier Bardem caricato. Daniel Craig ruim como...,0,0,3.5,6 anos atrás,20-04-2019 18:49,2.0,0.0,0.0,NaN,1.0,2489
6141,6141,movies,10-coisas-que-eu-odeio-em-voce-t1964,0,ADORO!!,0,0,4.5,8 anos atrás,20-04-2019 01:50,0.0,0.0,0.0,NaN,1.0,6141
7108,7108,movies,12-anos-de-escravidao-t43448,0,Existem certos filmes que funcionam como ápice...,2,0,5.0,5 anos atrás,18-04-2019 23:19,0.0,1.0,0.0,NaN,1.0,7108
14078,14078,movies,127-horas-t24843,0,"O filme é do James Franco, sem dúvidas. Se não...",0,0,4.5,8 anos atrás,20-04-2019 21:39,0.0,0.0,0.0,NaN,1.0,14078


In [117]:
## Definindo que todos os filmes que tem likes = 0, tem classificação negativa e os que possuem likes > 0, tem classificação positiva. Então, será criada uma nova feature para informar se a classificação é positiva (1) ou negativa (0)
df_teste= df_util.likes.replace((df_util != 0), 1)
df_teste.head(5)

ValueError: Series.replace cannot use dict-like to_replace and non-None value

# Trabalhando os dados

In [108]:
# Fazendo uma cópia dos dados, e selecionando as features que serão utilizadas
df = df_util.copy()

In [110]:
# selecionando as features de interesse
df = df[['object','text', 'likes']]
df.head(3)

,object,text,likes
0,007-cassino-royale-t23,Um dos melhores do 007,0
2489,007-operacao-skyfall-t31713,Javier Bardem caricato. Daniel Craig ruim como...,0
6141,10-coisas-que-eu-odeio-em-voce-t1964,ADORO!!,0


In [112]:
df.shape

(2442966, 3)

In [11]:
import time
start = time.time()
df["text_tokenizado"] = df["text"].astype(str).apply(nltk.word_tokenize)
print ("series.apply", (time.time() - start))

series.apply 9564.877822875977


AttributeError: 'str' object has no attribute 'astype'

# outra forma de tokenizar
start = time.time()
df["text_tokenizado2"] = df.apply(lambda row: nltk.word_tokenize(row["text"].astype(str)), axis=1)
print ("dataframe.apply", (time.time() - start))

# 3a forma de tokenizar o texto
from nltk import word_tokenize
tokens = word_tokenize(df['text'])
tokens

In [15]:
df

,text,likes,unlikes,text_tokenizado
0,Um dos melhores do 007,0,0,"[Um, dos, melhores, do, 007]"
1,"assisti só pela metade..do meio ao fim, o film...",0,0,"[assisti, só, pela, metade, .., do, meio, ao, ..."
2,foi um dos filmes mais violentos q já vi. mas ...,0,0,"[foi, um, dos, filmes, mais, violentos, q, já,..."
3,Adoroo esse filme!,0,0,"[Adoroo, esse, filme, !]"
4,O Daniel Crieg tem carisma zero como 007...,1,0,"[O, Daniel, Crieg, tem, carisma, zero, como, 0..."
...,...,...,...,...
2881584,"O App é ótimo! Porém, esses anúncios que apare...",79,0,"[O, App, é, ótimo, !, Porém, ,, esses, anúncio..."
2881585,O App é ótimo e tenho ele a mais de 1 ano! Mas...,9,0,"[O, App, é, ótimo, e, tenho, ele, a, mais, de,..."
2881586,"Eu super recomendo. Eu acho ótimo esse app,voc...",3,0,"[Eu, super, recomendo, ., Eu, acho, ótimo, ess..."
2881587,poderiam fazer uma atualização que permita bot...,1,0,"[poderiam, fazer, uma, atualização, que, permi..."


In [18]:
# salvando num dataframe
# salvando o dataset num CSV      
df.to_csv(r"C:\Users\Herica\Documents\Herica\Pos\GitHub\base\NLP\df_util_tokenizado.csv",index=False)

In [3]:
# abrindo o arquivo TOKENIZADO
df_tokenizado = pd.read_csv(r'C:\Users\Herica\Documents\Herica\Pos\GitHub\base\NLP\df_util_tokenizado.csv')
df_tokenizado.head(3)

,text,likes,unlikes,text_tokenizado
0,Um dos melhores do 007,0,0,"['Um', 'dos', 'melhores', 'do', '007']"
1,"assisti só pela metade..do meio ao fim, o film...",0,0,"['assisti', 'só', 'pela', 'metade', '..', 'do'..."
2,foi um dos filmes mais violentos q já vi. mas ...,0,0,"['foi', 'um', 'dos', 'filmes', 'mais', 'violen..."


In [7]:
print (df_tokenizado['text'][1])
print ('\n',df_tokenizado['text_tokenizado'][1])

assisti só pela metade..do meio ao fim, o filme parece ser mt bom ! espero poder assistir o filme inteiro logo logo!

 ['assisti', 'só', 'pela', 'metade', '..', 'do', 'meio', 'ao', 'fim', ',', 'o', 'filme', 'parece', 'ser', 'mt', 'bom', '!', 'espero', 'poder', 'assistir', 'o', 'filme', 'inteiro', 'logo', 'logo', '!']


# **2 - Visualização dos dados**
Coloque nesta seção os gráficos do PCA e do t-SNE, para cada representação. 

Responda também às seguintes perguntas: 

a) Existe algum padrão com relação às classes? 

b) Caso exista algum padrão, você pode concluir alguma coisa? 

c) Caso não exista, você consegue dizer se isso tem a ver com alguma representação ou classe?


## Divisão dos dados em treino e teste

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## Representação TF-IDF
Você pode usar tanto o gensim quanto o scikit para montar esta representação, mas lembre-se que é importante fazer o pré-processamento dos textos.


## Representação com o word2vec
O modelo poderá ser o apresentado na aula 03 ou algum outro modelo pré-treinado como os existentes no repositório http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc . Neste caso, cada documento deverá ser representado pelo vetor que resultar da média dos vetores de todas as palavras que o compõem. Em outras palavras, se D é composto pelas palavras w1, w2, …, wn, e seus vetores embeddings são v1, v2, …, vn, então a representação do documento de D será v = (v1 + v2 + … + vn) / n. 


## Extração de features do texto
Você deve pensar em ao menos 10 features para extrair do documento e que o possam representar. Aqui vão algumas sugestões: número de palavras, número de verbos, número de conjunções, número de palavras negativas, número de palavras fora do vocabulário, quantidades de entidades do tipo PESSOA, quantidade de entidades do tipo LOCAL, etc.


## Visualização dos seus dados com PCA

## Visualização dos seus dados com t-SNE

# **3 - Classificadores**
Descreva sucintamente os dois classificadores escolhidos (k-nn, SVM,RandomForest...). Você usou algum parâmetro que não seja padrão? Se sim, mencione nesta seção. 


# **4 - Resultados**
Escreva código que execute a validação cruzada em 5-folds para os dois classificadores escolhidos. Também responda às seguintes perguntas: Os embeddings realmente mostraram um resultado melhor que o TF-IDF? Se não, qual foi a representação que teve o melhor desempenho? A diferença foi muito grande?

# **5 - Conclusão**
Por fim fale aqui o que você conclui das visualizações e dos resultados. Tente explicar em detalhes por que um resultado, na sua opinião, foi melhor do que outro. Esta explicação pode incluir hipóteses para resultados melhores ou resultados piores. Também pode falar das dificuldades enfrentadas durante o trabalho e como conseguiu contorná-las.
